# Working with pandas: merging, apply, and groupby

**Description**  
This notebook demonstrates how to combine and manipulate tabular data with pandas.  
It covers the most commonly used methods for joining DataFrames (`merge`), applying functions to columns (`apply`), and grouping data (`groupby`).  
All examples use synthetic data created directly within the notebook for demonstration purposes.

**Learning goals**  
After completing this notebook, you will be able to:
- Merge pandas DataFrames using different join types (`left`, `right`, `inner`, `outer`)  
- Apply custom functions to DataFrames or columns using `apply`  
- Aggregate and summarize data with `groupby`  
- Understand how missing values (`NaN`) are introduced and handled during merging  
- Interpret merge results and debug mismatched keys  

**Authors**  
 
- Till Sauerwein, ORCID: [https://orcid.org/0000-0001-5830-4208](https://orcid.org/0000-0001-5830-4208)  
- Rabea Müller, ORCID: [https://orcid.org/0000-0002-3096-8237](https://orcid.org/0000-0002-3096-8237)  

**Affiliation**  
Data Science and Services, ZB MED – Information Centre for Life Sciences, Cologne, NRW, Germany  
Correspondence: [muellerr@zbmed.de](mailto:muellerr@zbmed.de)

**Date**  
Created: 2021-05-07  
Last updated: 2025-10-23  

**Requirements**  
- Python ≥ 3.10  
- pandas ≥ 2.1   
- Jupyter Notebook or JupyterLab environment  

**Licence for this notebook**  
Creative Commons Attribution 4.0 International (CC BY 4.0)  
https://creativecommons.org/licenses/by/4.0/


# Merging Dataframes

We create two DataFrames one containing DOIs and journal names, the other containing DOIs and first author names. Some DOIs appear in both DataFrames, others in only one. We will merge the DataFrames based on the DOI column.

In [1]:
import pandas as pd

In [2]:
doi_df_journal = pd.DataFrame({
    "DOI": [
        "10.1371/journal.pcbi.1004668",
        "10.21105/joss.01035",
        "10.12688/f1000research.18866.2",
        "10.1038/s41598-019-52881-4",
        "10.1186/s12859-019-3171-0"],
    "journal": [
        "PLoS Comput Biol",
        "JOSS",
        "F1000Res",
        "Sci Rep",
        "BMC Bioinformatics"]})

doi_df_first_author = pd.DataFrame({
    "DOI": [
        "10.1371/journal.pcbi.1004668",
        "10.21105/joss.01035",
        "10.21105/joss.01006",
        "10.12688/f1000research.18866.2",
        "10.1186/s12859-019-3171-0"],
    "first_author": [
        "Blischak",
        "Sparks",
        "Granger",
        "Thang",
        "Chen"]})

In [3]:
doi_df_first_author

,DOI,first_author
0,10.1371/journal.pcbi.1004668,Blischak
1,10.21105/joss.01035,Sparks
2,10.21105/joss.01006,Granger
3,10.12688/f1000research.18866.2,Thang
4,10.1186/s12859-019-3171-0,Chen


In [4]:
doi_df_journal

,DOI,journal
0,10.1371/journal.pcbi.1004668,PLoS Comput Biol
1,10.21105/joss.01035,JOSS
2,10.12688/f1000research.18866.2,F1000Res
3,10.1038/s41598-019-52881-4,Sci Rep
4,10.1186/s12859-019-3171-0,BMC Bioinformatics


### Left merge
Includes all DOIs that appear in the left (first) DataFrame. If a DOI is missing from the right DataFrame, its corresponding columns will contain `NaN` values.

In [5]:
doi_df_first_author.merge(doi_df_journal, on="DOI", how="left")

,DOI,first_author,journal
0,10.1371/journal.pcbi.1004668,Blischak,PLoS Comput Biol
1,10.21105/joss.01035,Sparks,JOSS
2,10.21105/joss.01006,Granger,NaN
3,10.12688/f1000research.18866.2,Thang,F1000Res
4,10.1186/s12859-019-3171-0,Chen,BMC Bioinformatics


In this case, the journal column shows `NaN` for Granger, and the journal *Sci Rep* is not displayed, since its DOI appears only in the right DataFrame.

### Right merge
All DOIs located in the right-hand or second data frame are listed.

If the left-hand or first data frame does not contain the DOI of the right-hand data frame, `NaN` is inserted in the columns of the left-hand data frame instead of a value.

In [6]:
doi_df_first_author.merge(doi_df_journal, on="DOI", how="right")

,DOI,first_author,journal
0,10.1371/journal.pcbi.1004668,Blischak,PLoS Comput Biol
1,10.21105/joss.01035,Sparks,JOSS
2,10.12688/f1000research.18866.2,Thang,F1000Res
3,10.1038/s41598-019-52881-4,NaN,Sci Rep
4,10.1186/s12859-019-3171-0,Chen,BMC Bioinformatics


Now we see a `NaN` for the first author for the journal *Sci Rep*, and Granger is no longer displayed because it does not appear in the second DataFrame.

### Inner merge
Only entries that appear in both original DataFrames (matching DOIs) are included in the new DataFrame.

In [7]:
doi_df_first_author.merge(doi_df_journal, on="DOI", how="inner")

,DOI,first_author,journal
0,10.1371/journal.pcbi.1004668,Blischak,PLoS Comput Biol
1,10.21105/joss.01035,Sparks,JOSS
2,10.12688/f1000research.18866.2,Thang,F1000Res
3,10.1186/s12859-019-3171-0,Chen,BMC Bioinformatics


`Granger` and `Sci Rep` are excluded.

### Outer merge
All DOIs from both DataFrames are included. Missing values are filled with `NaN`.

In [8]:
doi_df_first_author.merge(doi_df_journal, on="DOI", how="outer")

,DOI,first_author,journal
0,10.1371/journal.pcbi.1004668,Blischak,PLoS Comput Biol
1,10.21105/joss.01035,Sparks,JOSS
2,10.21105/joss.01006,Granger,NaN
3,10.12688/f1000research.18866.2,Thang,F1000Res
4,10.1186/s12859-019-3171-0,Chen,BMC Bioinformatics
5,10.1038/s41598-019-52881-4,NaN,Sci Rep


## Apply
A function is applied to the entire DataFrame or a single column.

We create a DataFrame with inconsistent author names.

In [9]:
doi_df_first_author_2 = pd.DataFrame({
    "DOIs": [
        "10.1371/journal.pcbi.1004668",
        "10.21105/joss.01035",
        "10.21105/joss.01006",
        "10.12688/f1000research.18866.2",
        "10.1186/s12859-019-3171-0"],
    "first_author": [
        "Blischak J.",
        "Sparks Adam H.",
        "Granger Nicolas",
        "Thang",
        "Chen Danze"]})

In [10]:
doi_df_first_author_2

,DOIs,first_author
0,10.1371/journal.pcbi.1004668,Blischak J.
1,10.21105/joss.01035,Sparks Adam H.
2,10.21105/joss.01006,Granger Nicolas
3,10.12688/f1000research.18866.2,Thang
4,10.1186/s12859-019-3171-0,Chen Danze


## The split function (method)

In [11]:
# With split, we can separate a contiguous string at a specific point and convert it into a list.
"Rosalind Franklin".split(" ")

['Rosalind', 'Franklin']

In [12]:
# We can then address a position from this list. 
"Rosalind Franklin".split(" ")[0]

'Rosalind'

### The “split” function is applied to a column.
First, let's separate the authors' first names from their surnames.

In [13]:
doi_df_first_author_2["first_author"].apply(lambda author: author.split(" ")[0])

0    Blischak
1      Sparks
2     Granger
3       Thang
4        Chen
Name: first_author, dtype: object

.apply(lambda author: author.split(‘ ’)[0]): This part of the expression applies the apply function to the selected column. The apply function allows you to apply a function to each row or element in the selected column.

lambda author: author.split(‘ ’)[0]: Here, an anonymous function (lambda function) is defined that is applied to each element (in this case, the author name) in the ‘first_author’ column. This function splits the author name at the space (at the spaces) and returns the first element of the resulting split list, which represents the author's first name.

The split(‘ ’)[0] part splits the name into words (using the space) and returns the first word (the first name).

### The result (the surname) is assigned to a column.
To store the result in a new column, we can define it as a column and assign it a value using the function.

In [14]:
doi_df_first_author_2["last_name"] = doi_df_first_author_2["first_author"].apply(lambda author: author.split(" ")[0])

In [15]:
doi_df_first_author_2

,DOIs,first_author,last_name
0,10.1371/journal.pcbi.1004668,Blischak J.,Blischak
1,10.21105/joss.01035,Sparks Adam H.,Sparks
2,10.21105/joss.01006,Granger Nicolas,Granger
3,10.12688/f1000research.18866.2,Thang,Thang
4,10.1186/s12859-019-3171-0,Chen Danze,Chen


The surnames are now stored in a new column, but are still in the first_author column. Now we want to filter out the first names by addressing the second position.

In [16]:
doi_df_first_author_2["first_author"].apply(lambda author: author.split(" ")[1:])

0          [J.]
1    [Adam, H.]
2     [Nicolas]
3            []
4       [Danze]
Name: first_author, dtype: object

Our goal is to display the first names so that the initial of the first name is output and written in a new column. This means we first have to clean up the first names using a function, as they are currently different.

In [17]:
def first_name_initial(full_name):
    return full_name.split(" ")[1:]

The function is then passed to apply.

In [18]:
doi_df_first_author_2["first_author"].apply(first_name_initial)

0          [J.]
1    [Adam, H.]
2     [Nicolas]
3            []
4       [Danze]
Name: first_author, dtype: object

The function can now separate the surname. Now we extend the function so that we only get back the initial of the first first name. If no first name exists, a NaN is returned.

In [19]:
first_name = "Sparks Adam, H.".split(" ")[1:]   

In [20]:
first_name[0]

'Adam,'

In [21]:
first_name[0][0]

'A'

In [22]:
first_name[0][0] + "."

'A.'

In [24]:
def first_name_initial(full_name):
    # Split the full name into parts
    parts = full_name.split(' ')
    
    # If the name has at least two parts (first and last name)
    if len(parts) > 1:
        # Take the first letter of the first name and add a dot
        return parts[0][0] + '.'
    else:
        # If no first name could be extracted, return 'NaN'
        return 'NaN'


In [25]:
### The function is passed to apply again
doi_df_first_author_2["first_author"].apply(first_name_initial)

0     B.
1     S.
2     G.
3    NaN
4     C.
Name: first_author, dtype: object

### The initial is stored in a new column.

In [26]:
doi_df_first_author_2["first_name"] = doi_df_first_author_2["first_author"].apply(first_name_initial)

In [27]:
doi_df_first_author_2

,DOIs,first_author,last_name,first_name
0,10.1371/journal.pcbi.1004668,Blischak J.,Blischak,B.
1,10.21105/joss.01035,Sparks Adam H.,Sparks,S.
2,10.21105/joss.01006,Granger Nicolas,Granger,G.
3,10.12688/f1000research.18866.2,Thang,Thang,NaN
4,10.1186/s12859-019-3171-0,Chen Danze,Chen,C.


### Apply is applied line by line to the entire data frame
Now let's use first_name and last_name to introduce a new column in which the cleaned author names are specified. 
To do this, we combine the columns first_name and last_name with a space in a new column. Here, too, we can use the anonymous function lambda again

In [28]:
doi_df_first_author_2["first_author_cleaned"] = doi_df_first_author_2.apply(
    lambda df: df["last_name"] + " " + df["first_name"], axis=1)

axis=1: This parameter specifies that the apply function should be applied row by row. This means that the lambda function is executed for each row in the DataFrame, and the result is stored in the new column ‘first_author_cleaned’ in the same row.

In [29]:
doi_df_first_author_2

,DOIs,first_author,last_name,first_name,first_author_cleaned
0,10.1371/journal.pcbi.1004668,Blischak J.,Blischak,B.,Blischak B.
1,10.21105/joss.01035,Sparks Adam H.,Sparks,S.,Sparks S.
2,10.21105/joss.01006,Granger Nicolas,Granger,G.,Granger G.
3,10.12688/f1000research.18866.2,Thang,Thang,NaN,Thang NaN
4,10.1186/s12859-019-3171-0,Chen Danze,Chen,C.,Chen C.


It looks almost perfect. The only thing missing is that the NaN values are processed correctly.
To do this, we add another condition.

In [30]:
doi_df_first_author_2["first_author_cleaned"] = doi_df_first_author_2.apply(
    lambda df: df["last_name"] + " " + df["first_name"] if df["first_name"] != "NaN" else df["last_name"], axis=1)

With the condition that the values should be linked if no NaN is contained. Otherwise, only the surname should be stored in the column.

In [31]:
doi_df_first_author_2

,DOIs,first_author,last_name,first_name,first_author_cleaned
0,10.1371/journal.pcbi.1004668,Blischak J.,Blischak,B.,Blischak B.
1,10.21105/joss.01035,Sparks Adam H.,Sparks,S.,Sparks S.
2,10.21105/joss.01006,Granger Nicolas,Granger,G.,Granger G.
3,10.12688/f1000research.18866.2,Thang,Thang,NaN,Thang
4,10.1186/s12859-019-3171-0,Chen Danze,Chen,C.,Chen C.


## Groupby

With the groupby function in Pandas, we can group data based on one or more columns and then apply various aggregation functions to these groups. 

In [32]:

df_doi_journal_price = pd.DataFrame({
    "DOI": [
        "10.1371/journal.pcbi.1004668",
        "10.21105/joss.01035",
        "10.12688/f1000research.18866.2",
        "10.1038/s41598-019-52881-4",
        "10.1186/s12859-019-3171-0",
        "10.1186/s12859-019-3171-1",
        "10.1186/s12859-019-3171-3"],
    "journal": [
        "PLoS Comput Biol",
        "BMC Bioinformatics",
        "F1000Res",
        "Sci Rep",
        "BMC Bioinformatics",
        "PLoS Comput Biol",
        "BMC Bioinformatics"],
    "price": [
        2000,
        2000,
        1500,
        1000,
        1500,
        1500,
        1600
    ]})

In [33]:
df_doi_journal_price

,DOI,journal,price
0,10.1371/journal.pcbi.1004668,PLoS Comput Biol,2000
1,10.21105/joss.01035,BMC Bioinformatics,2000
2,10.12688/f1000research.18866.2,F1000Res,1500
3,10.1038/s41598-019-52881-4,Sci Rep,1000
4,10.1186/s12859-019-3171-0,BMC Bioinformatics,1500
5,10.1186/s12859-019-3171-1,PLoS Comput Biol,1500
6,10.1186/s12859-019-3171-3,BMC Bioinformatics,1600


Create a groupby object. We want to group the DataFrame df_doi_journal_price according to the column ‘journal’. This means that all rows with the same value in the column ‘journal’ are combined into one group. In this case, the articles are grouped by journal.

In [34]:
grp_journal = df_doi_journal_price.groupby(["journal"])

In [35]:
grp_journal

In Pandas, the GroupBy object grp_journal is not automatically displayed when we evaluate it. Instead, we must apply an aggregation function or method to display or use the results of the grouping.

In [36]:
grp_journal.apply(display)

,DOI,journal,price
1,10.21105/joss.01035,BMC Bioinformatics,2000
4,10.1186/s12859-019-3171-0,BMC Bioinformatics,1500
6,10.1186/s12859-019-3171-3,BMC Bioinformatics,1600


,DOI,journal,price
2,10.12688/f1000research.18866.2,F1000Res,1500


,DOI,journal,price
0,10.1371/journal.pcbi.1004668,PLoS Comput Biol,2000
5,10.1186/s12859-019-3171-1,PLoS Comput Biol,1500


,DOI,journal,price
3,10.1038/s41598-019-52881-4,Sci Rep,1000


""


We could now apply aggregation functions to our groups. For example, we want to see the average price for each journal.

In [37]:
grp_journal["price"].mean()

journal
BMC Bioinformatics    1700.0
F1000Res              1500.0
PLoS Comput Biol      1750.0
Sci Rep               1000.0
Name: price, dtype: float64

Oder die Summe:

In [38]:
grp_journal["price"].sum()

journal
BMC Bioinformatics    5100
F1000Res              1500
PLoS Comput Biol      3500
Sci Rep               1000
Name: price, dtype: int64

## Summary

In this notebook you have learned how to:
- Merge pandas DataFrames using different join types (`left`, `right`, `inner`, `outer`)
- Apply custom functions to columns and DataFrames using `apply`
- Aggregate and summarize data using `groupby`

These are some of the most powerful tools for data wrangling in pandas and form the foundation for most data analysis workflows in Python.

---

## Further reading

- [pandas documentation – merge, join, concatenate and compare](https://pandas.pydata.org/docs/user_guide/merging.html)
- [pandas documentation – apply](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html)
- [pandas documentation – groupby](https://pandas.pydata.org/docs/user_guide/groupby.html)
- [Python Data Science Handbook by Jake VanderPlas](https://jakevdp.github.io/PythonDataScienceHandbook/)

---

## Acknowledgements

This notebook was created for educational purposes using examples inspired by the pandas documentation.  
No external data sources were used.

---

## Citation and licence

If you use or adapt this notebook, please cite it as follows:

> Sauerwein, T.; Müller, R. (2025). *Working with pandas: merging, apply, and groupby* [Jupyter Notebook].  
> Creative Commons Attribution 4.0 International (CC BY 4.0).

Licence: [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/)
